<a href="https://colab.research.google.com/github/the-faisalahmed/Optimization/blob/main/Scales_and_Weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

!pip install pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.util.infeasible import log_infeasible_constraints
from pyomo.opt import SolverStatus
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
from pyomo.contrib.latex_printer import latex_printer


In [2]:
def SolveModel(model, solver):
  # Solve model
  opt = SolverFactory(solver)
  result = opt.solve(model)

  if (result.solver.status == SolverStatus.ok) and \
      (result.solver.termination_condition == TerminationCondition.optimal):
      # Do something when the solution in optimal and feasible
      print('Solution is Optimal')
  elif (result.solver.termination_condition == TerminationCondition.infeasible):
      # Do something when model in infeasible
      print('Solution is Infeasible')
  else:
          # Something else is wrong
      print("Solver Status:",  result.solver.status)

  # Solve time
  print('Solve Time: ', result.solver.wallclock_time)

In [45]:
def ScaleWeightModel(n, weights):
  model = ConcreteModel()

  model.X = Var(RangeSet(n), RangeSet(weights), within = Boolean)
  model.Y = Var(RangeSet(weights), bounds = (1,n), within = PositiveIntegers)

  # Binary variable selecting weights per scale
  def con1_rule(model, i):
    return sum(model.X[i,j]*model.Y[j] for j in RangeSet(weights)) == i
  model.con1 = Constraint(RangeSet(n), rule = con1_rule)

  SolveModel(model, 'couenne')

  # Solution
  sol = sorted([int(value(model.Y[i])) for i in RangeSet(weights)])

  print('\n')
  print('Solution:')
  print(sol)

ScaleWeightModel(n= 15, weights= 4)

Solution is Optimal
Solve Time:  <undefined>


Solution:
[1, 2, 4, 8]
